# Imports

In [71]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import math
import time

import warnings
warnings.filterwarnings('ignore')

from sklearnex import patch_sklearn
patch_sklearn()

# from lazypredict.Supervised import LazyRegressor

# Visualization
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Feature and Model Selection
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Models
# Linear
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import Lars
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLars
from sklearn.linear_model import LassoLarsCV
from sklearn.linear_model import LassoLarsIC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LarsCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

# Ensemble
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

# SVM
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.svm import LinearSVR

# Other
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.dummy import DummyRegressor
from sklearn.kernel_ridge import KernelRidge

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# Data Loading and Preparation

In [72]:
# !wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
# !wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv
# !wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv

In [73]:
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

## Exploring Train Data

In [74]:
train_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,7,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,...,3,3,3,[1-100],50000.0,2,0,0,0,0
1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,12,12,[1-100],10000.0,5,0,0,0,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28,28,28,[1-100],10000.0,2,0,0,0,0
3,22,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,...,8,8,8,[1-100],300000.0,1,0,0,0,0
4,23,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,...,27,27,27,[1-100],10000.0,2,0,0,0,0


In [75]:
print('Number of rows in train_df: ', train_df.shape[0])
print('Number of columns in train_df: ', train_df.shape[1])
print('Number of values in train_df: ', train_df.count().sum())
print('Number of NaNs in train_df: ', sum(train_df.isna().sum()))
print('Number of NaNs in train_df in all columns:\n')
print(train_df.isna().sum())

Number of rows in train_df:  32395
Number of columns in train_df:  37
Number of values in train_df:  835135
Number of NaNs in train_df:  363480
Number of NaNs in train_df in all columns:

record_id                           0
ar_revenue                      15145
ar_total_expenses               15145
ar_sale_cost                    15145
ar_selling_expenses             15145
ar_management_expenses          15145
ar_sale_profit                  15145
ar_balance_of_rvns_and_expns    15145
ar_profit_before_tax            15145
ar_taxes                        15145
ar_other_profit_and_losses      15145
ar_net_profit                   15145
ab_immobilized_assets           15145
ab_mobile_current_assets        15145
ab_inventory                    15145
ab_accounts_receivable          15145
ab_other_current_assets         15145
ab_cash_and_securities          15145
ab_losses                       15145
ab_own_capital                  15145
ab_borrowed_capital             15145
ab_long_term_l

In [76]:
train_df.describe()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
count,32395.000000,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,...,32395.000000,32395.000000,32395.000000,32395.000000,3.239500e+04,32395.000000,32395.000000,32395.000000,32395.0,32395.000000
mean,136955.533601,8.153776e+07,7.678474e+07,7.320328e+07,2.499198e+06,1.082263e+06,4.753016e+06,-6.424773e+05,4.110538e+06,4.188071e+05,...,78.644822,15.010341,15.010341,15.010341,1.366509e+06,11.946535,0.021670,0.000093,0.0,0.064547
std,79370.573366,1.274402e+08,1.227227e+08,1.156651e+08,1.231798e+07,7.848929e+06,1.941027e+07,6.014715e+06,1.938021e+07,1.565298e+06,...,46.522534,9.546511,9.546511,9.546511,1.502438e+07,86.341847,0.342715,0.009623,0.0,0.245729
min,7.000000,0.000000e+00,-7.019840e+08,-5.072780e+08,-3.542900e+07,-1.921670e+08,-9.210000e+07,-7.581800e+07,-5.946600e+07,-1.681000e+07,...,0.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000,0.000000,0.0,0.000000
25%,67912.000000,1.019950e+07,9.130250e+06,8.844000e+06,0.000000e+00,0.000000e+00,2.112500e+05,-9.400000e+05,1.420000e+05,0.000000e+00,...,35.000000,6.000000,6.000000,6.000000,1.000000e+04,1.000000,0.000000,0.000000,0.0,0.000000
50%,137527.000000,3.233500e+07,2.970350e+07,2.840800e+07,0.000000e+00,0.000000e+00,1.231500e+06,-1.260000e+05,8.500000e+05,4.100000e+04,...,77.000000,14.000000,14.000000,14.000000,1.000000e+04,1.000000,0.000000,0.000000,0.0,0.000000
75%,205633.500000,8.887675e+07,8.309500e+07,7.991425e+07,0.000000e+00,0.000000e+00,4.574750e+06,0.000000e+00,3.543750e+06,2.910000e+05,...,133.000000,24.000000,24.000000,24.000000,3.000000e+04,2.000000,0.000000,0.000000,0.0,0.000000
max,274623.000000,7.946840e+08,7.949360e+08,7.624930e+08,2.746560e+08,3.023550e+08,1.434511e+09,1.584250e+08,1.481526e+09,5.420200e+07,...,149.000000,33.000000,33.000000,33.000000,1.584979e+09,998.000000,20.000000,1.000000,0.0,1.000000


In [77]:
train_df.dtypes

record_id                         int64
ar_revenue                      float64
ar_total_expenses               float64
ar_sale_cost                    float64
ar_selling_expenses             float64
ar_management_expenses          float64
ar_sale_profit                  float64
ar_balance_of_rvns_and_expns    float64
ar_profit_before_tax            float64
ar_taxes                        float64
ar_other_profit_and_losses      float64
ar_net_profit                   float64
ab_immobilized_assets           float64
ab_mobile_current_assets        float64
ab_inventory                    float64
ab_accounts_receivable          float64
ab_other_current_assets         float64
ab_cash_and_securities          float64
ab_losses                       float64
ab_own_capital                  float64
ab_borrowed_capital             float64
ab_long_term_liabilities        float64
ab_short_term_borrowing         float64
ab_accounts_payable             float64
ab_other_borrowings             float64


## Exploring Test Data

In [78]:
test_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg
0,196,39103000.0,38913000.0,38913000.0,0.0,0.0,190000.0,141000.0,331000.0,284000.0,...,135,3,3,3,[1-100],100000.0,1,0,0,0
1,1196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,138,5,5,5,[1-100],100000.0,1,0,0,0
2,2813,54174000.0,50929000.0,50929000.0,0.0,0.0,3245000.0,-1278000.0,1967000.0,500000.0,...,138,20,20,20,[1-100],250000.0,2,0,0,0
3,4385,1904000.0,1679000.0,1679000.0,0.0,0.0,225000.0,-88000.0,137000.0,0.0,...,139,9,9,9,[1-100],10000.0,2,0,0,0
4,6479,225584000.0,210685000.0,210685000.0,0.0,0.0,14899000.0,-12715000.0,2184000.0,445000.0,...,139,3,3,3,(100-500],1700598.0,84,0,0,0


In [79]:
print('Number of rows in test_df: ', test_df.shape[0])
print('Number of columns in test_df: ', test_df.shape[1])
print('Number of values in test_df: ', test_df.count().sum())
print('Number of NaNs in test_df: ', sum(test_df.isna().sum()))
print('Number of NaNs in test_df in all columns:\n')
print(test_df.isna().sum())

Number of rows in test_df:  200
Number of columns in test_df:  36
Number of values in test_df:  4368
Number of NaNs in test_df:  2832
Number of NaNs in test_df in all columns:

record_id                         0
ar_revenue                      118
ar_total_expenses               118
ar_sale_cost                    118
ar_selling_expenses             118
ar_management_expenses          118
ar_sale_profit                  118
ar_balance_of_rvns_and_expns    118
ar_profit_before_tax            118
ar_taxes                        118
ar_other_profit_and_losses      118
ar_net_profit                   118
ab_immobilized_assets           118
ab_mobile_current_assets        118
ab_inventory                    118
ab_accounts_receivable          118
ab_other_current_assets         118
ab_cash_and_securities          118
ab_losses                       118
ab_own_capital                  118
ab_borrowed_capital             118
ab_long_term_liabilities        118
ab_short_term_borrowing        

In [80]:
test_df.describe()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg
count,200.000000,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,...,200.000000,200.000000,200.000000,200.000000,200.000000,2.000000e+02,200.000000,200.0,200.0,200.0
mean,135340.925000,8.625971e+07,8.200218e+07,7.694863e+07,2.780183e+06,2.273366e+06,4.257524e+06,-6.880854e+05,3.569439e+06,4.419878e+05,...,84.595000,65.680000,14.320000,14.320000,14.320000,7.263560e+05,5.070000,0.0,0.0,0.0
std,80177.086856,1.589030e+08,1.534092e+08,1.372651e+08,1.180692e+07,1.752820e+07,7.752518e+06,5.253569e+06,8.232787e+06,1.294710e+06,...,80.585115,46.096442,9.697308,9.697308,9.697308,4.043191e+06,32.669582,0.0,0.0,0.0
min,196.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.898000e+06,-2.328100e+07,-6.390000e+06,-1.100000e+04,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.0,0.0,0.0
25%,68367.500000,6.779250e+06,5.397500e+06,5.397500e+06,0.000000e+00,0.000000e+00,1.557500e+05,-6.570000e+05,1.265000e+05,0.000000e+00,...,25.000000,25.000000,5.000000,5.000000,5.000000,1.000000e+04,1.000000,0.0,0.0,0.0
50%,127585.500000,1.899700e+07,1.645600e+07,1.587850e+07,0.000000e+00,0.000000e+00,1.184000e+06,-5.800000e+04,5.450000e+05,7.350000e+04,...,57.000000,54.500000,12.500000,12.500000,12.500000,1.000000e+04,1.000000,0.0,0.0,0.0
75%,202665.000000,8.388250e+07,7.831350e+07,7.651175e+07,0.000000e+00,0.000000e+00,3.200000e+06,0.000000e+00,2.630500e+06,3.525000e+05,...,106.000000,101.250000,24.000000,24.000000,24.000000,2.055575e+04,2.000000,0.0,0.0,0.0
max,271584.000000,7.707240e+08,7.458810e+08,6.696010e+08,9.397600e+07,1.564560e+08,3.805100e+07,2.858600e+07,5.448100e+07,7.690000e+06,...,460.000000,145.000000,33.000000,33.000000,33.000000,4.017000e+07,420.000000,0.0,0.0,0.0


In [81]:
test_df.dtypes

record_id                         int64
ar_revenue                      float64
ar_total_expenses               float64
ar_sale_cost                    float64
ar_selling_expenses             float64
ar_management_expenses          float64
ar_sale_profit                  float64
ar_balance_of_rvns_and_expns    float64
ar_profit_before_tax            float64
ar_taxes                        float64
ar_other_profit_and_losses      float64
ar_net_profit                   float64
ab_immobilized_assets           float64
ab_mobile_current_assets        float64
ab_inventory                    float64
ab_accounts_receivable          float64
ab_other_current_assets         float64
ab_cash_and_securities          float64
ab_losses                       float64
ab_own_capital                  float64
ab_borrowed_capital             float64
ab_long_term_liabilities        float64
ab_short_term_borrowing         float64
ab_accounts_payable             float64
ab_other_borrowings             float64


# Data Preprocessing and Feature Selection

In [82]:
# MinMaxScaler for data normalization
# KNNImputer is a distance-based imputation method,
# so it needs normalization before imputing
scaler = MinMaxScaler()
record_id = train_df['record_id']
default = train_df['default_12m']
train_df = pd.DataFrame(scaler.fit_transform(train_df.drop(['default_12m', 'record_id', 'ul_staff_range'], axis=1)), columns = train_df.drop(['default_12m', 'record_id', 'ul_staff_range'], axis=1).columns)
train_df['record_id'] = record_id
train_df['default_12m'] = default
# df_normalized = scaler.fit_transform(df.drop(['target', 'id'], axis=1))

In [83]:
train_df = train_df.dropna(axis=0)

# Cross-Validation

In [84]:
# Get a list of models to evaluate
def get_models():
    models = dict()
    models['SGDRegressor'] = SGDRegressor()
    models['KernelRidge'] = KernelRidge()
    models['LinearSVR'] = LinearSVR()
    models['MLPRegressor'] = MLPRegressor()
    models['HuberRegressor'] = HuberRegressor()
    models['PassiveAggressiveRegressor'] = PassiveAggressiveRegressor()
    models['Lars'] = Lars()
    models['Lasso'] = Lasso()
    models['LassoLars'] = LassoLars()
    models['LassoLarsCV'] = LassoLarsCV()
    # models['LassoLarsIC'] = LassoLarsIC()
    models['GaussianProcessRegressor'] = GaussianProcessRegressor()
    models['TweedieRegressor'] = TweedieRegressor()
    models['OrthogonalMatchingPursuit'] = OrthogonalMatchingPursuit()
    models['ExtraTreesRegressor'] = ExtraTreesRegressor()
    models['DecisionTreeRegressor'] = DecisionTreeRegressor()
    models['TransformedTargetRegressor'] = TransformedTargetRegressor()
    models['KNeighborsRegressor'] = KNeighborsRegressor()
    models['GradientBoostingRegressor'] = GradientBoostingRegressor()
    models['SVR'] = SVR()
    models['NuSVR'] = NuSVR()
    models['RandomForestRegressor'] = RandomForestRegressor()
    models['LarsCV'] = LarsCV()
    models['LassoCV'] = LassoCV()
    models['BayesianRidge'] = BayesianRidge()
    models['ElasticNet'] = ElasticNet()
    models['ElasticNetCV'] = ElasticNetCV()
    models['DummyRegressor'] = DummyRegressor()
    models['AdaBoostRegressor'] = AdaBoostRegressor()
    models['BaggingRegressor'] = BaggingRegressor()
    models['Ridge'] = Ridge()
    models['RidgeCV'] = RidgeCV()
    models['Stacking'] = get_stacking()
    return models

In [85]:
# Evaluate a given model using cross-validation
def mape_evaluate_model(model, x, y):
    start_time = time.time()
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_percentage_error', cv=cv, n_jobs=-1, error_score='raise')
    run_time = time.time() - start_time
    return scores, run_time

In [86]:
# Get a stacking ensemble of models
def get_stacking():
    # Defining the base models
    level0 = list()
    level0.append(('SGDRegressor', SGDRegressor()))
    level0.append(('KernelRidge', KernelRidge()))
    level0.append(('LinearSVR', LinearSVR()))
    level0.append(('MLPRegressor', MLPRegressor()))
    # Defining meta learner model
    level1 = LinearRegression()
    # Defining the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return model

In [87]:
# Get the models to evaluate
models = get_models()
mape_results, names, runs = list(), list(), list()

# Evaluate the models and store results
for name, model in models.items():
    mape_scores, run_time = mape_evaluate_model(model, train_df.drop('default_12m', axis=1), train_df['default_12m'])
    mape_results.append(mape_scores)
    names.append(name)
    runs.append(run_time)
    print(name, "| MAPE:", mean(mape_scores), "| Standard Deviation:", std(mape_scores),"| Time Taken:", run_time)

SGDRegressor | MAPE: -1.3937763607235984e+34 | Standard Deviation: 8.184769680137398e+33 | Time Taken: 13.718936920166016


KeyboardInterrupt: 

# Model Selection

In [ ]:
mape_mean_results, mape_std_results = list(), list()
r2_mean_results, r2_std_results = list(), list()
for value in range(len(mape_results)):
    mape_mean_results.append(mean(mape_results[value]))
    mape_std_results.append(std(mape_results[value]))

In [ ]:
data = {'Name':names, 'MAPE':mape_mean_results, 'Standart Deviation':mape_std_results, 'Time Taken':runs}
df_models = pd.DataFrame(data)
df_models.set_index('Name', inplace=True)
df_models

In [ ]:
line = px.line(data_frame = df_models.sort_values('Time Taken', ascending=True),y =['Time Taken'], markers = True)
line.update_xaxes(title="Model", rangeslider_visible = False)
line.update_yaxes(title = "Time (s)")
line.update_traces(line_color="red")
line.update_layout(showlegend = True,
                   title = {
                       'text': 'Time Taken vs Model',
                       'y':0.94,
                       'x':0.5,
                       'xanchor': 'center',
                       'yanchor': 'top'})

line.show()

In [ ]:
line = px.line(data_frame = df_models.sort_values('MAPE', ascending=False),y =['MAPE'], markers = True)
line.update_xaxes(title="Model", rangeslider_visible = False)
line.update_yaxes(title = "MAPE Score")
line.update_traces(line_color="blue")
line.update_layout(showlegend = True,
                   title = {
                       'text': 'MAPE vs Model',
                       'y':0.94,
                       'x':0.5,
                       'xanchor': 'center',
                       'yanchor': 'top'})

line.show()

In [ ]:
line = px.line(data_frame = df_models.sort_values('Standart Deviation', ascending=True),y =['Standart Deviation'], markers = True)
line.update_xaxes(title="Model", rangeslider_visible = False)
line.update_yaxes(title = "Standart Deviation")
line.update_traces(line_color="purple")
line.update_layout(showlegend = True,
                   title = {
                       'text': 'Standart Deviation vs Model',
                       'y':0.94,
                       'x':0.5,
                       'xanchor': 'center',
                       'yanchor': 'top'})

line.show()

# Modeling

In [ ]:
# Define the base models
level0 = list()
level0.append(('SGDRegressor', SGDRegressor()))
level0.append(('KernelRidge', KernelRidge()))
level0.append(('LinearSVR', LinearSVR()))
level0.append(('MLPRegressor', MLPRegressor()))
# Define meta learner model
level1 = LinearRegression()
# Define the stacking ensemble
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

# Fit the model on all available data
model.fit(train_df.drop('target', axis=1), train_df['target'])
# Predict the results
predictions = model.predict(test_df.drop('target', axis=1))
print("Predicted Values:", predictions)